### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Chronic_obstructive_pulmonary_disease_(COPD)/GSE162635'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = True
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_age = None  # define the functions when applicable

# Check and assign trait_row for 'Chronic_obstructive_pulmonary_disease_(COPD)'
copd_keys = [2, 3, 4]
for key in copd_keys:
    unique_values = ['gold.1', 'gold.2', 'gold.3']  # All relate to COPD trait
    if any([key in item for item in sample_characteristics_dictionary[key]]):
        trait_row = key
        break

# Age and gender data are unavailable; setting up keys and functions
age_row = gender_row = None

# Data type conversion functions
def convert_trait(value):
    if ':' in value:
        trait_state = value.split(':')[1].strip()
        if trait_state.lower() in ['healthy', 'normal', 'control']:
            return 0
        elif trait_state.lower() in ['o', 'i', 'ii', 'iii', 'iv']:
            return 1  # Assuming these are stages of COPD
    return None

def convert_age(value):
    return None  # Age data is not available

def convert_gender(value):
    return None  # Gender data is not available

# Save cohort information
save_cohort_info('GSE162635', './preprocessed/Chronic_obstructive_pulmonary_disease_(COPD)/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Chronic_obstructive_pulmonary_disease_(COPD)', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Chronic_obstructive_pulmonary_disease_(COPD)/trait_data/GSE162635.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
